# Create Vector Embeddings

In [1]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer



c:\Users\Sanya Kulthia\OneDrive\Desktop\sans_bot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embed_model = SentenceTransformer(
    "paraphrase-multilingual-MiniLM-L12-v2"
)



In [3]:
chroma_client = chromadb.Client(
    Settings(
        persist_directory="./chroma_db",  # THIS is your vector DB
        anonymized_telemetry=False
    )
)


In [4]:
collection = chroma_client.get_or_create_collection(
    name="sanskrit_docs",
    metadata={"hnsw:space": "cosine"}
)


In [5]:
import json

with open("chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

len(chunks)


39

In [6]:
texts = [c["text"] for c in chunks]

metadatas = [
    {
        "chunk_id": c["chunk_id"],
        "language": c["language"],
        "script": c["script"],
        "section_id": c["section_id"],
        "position": c["position"]
    }
    for c in chunks
]

ids = [c["chunk_id"] for c in chunks]


In [7]:
embeddings = embed_model.encode(
    texts,
    convert_to_numpy=True,
    normalize_embeddings=True
)


In [8]:
collection.add(
    documents=texts,
    embeddings=embeddings.tolist(),
    metadatas=metadatas,
    ids=ids
)




In [9]:
collection.count()


39

In [10]:
def rerank(results):
    reranked = []

    for doc, meta, dist in zip(
        results["documents"][0],
        results["metadatas"][0],
        results["distances"][0]
    ):
        score = 1 - dist

        # Penalize very short chunks (titles)
        if len(doc) < 30:
            score *= 0.6

        reranked.append({
            "score": round(score, 3),
            "language": meta["language"],
            "text": doc,
            "section_id": meta["section_id"]
        })

    return sorted(reranked, key=lambda x: x["score"], reverse=True)


In [11]:
from collections import Counter

def filter_by_section(reranked, top_n=5):
    section_counts = Counter(r["section_id"] for r in reranked)
    best_section = section_counts.most_common(1)[0][0]

    return [
        r for r in reranked
        if r["section_id"] == best_section
    ][:top_n]



In [12]:
def extract_keywords(query):
    # keep long, distinctive tokens
    return [w for w in query.split() if len(w) >= 5]


In [13]:
def keyword_filter_raw(raw, keywords):
    if not keywords:
        return raw

    filtered_docs = []
    filtered_metas = []
    filtered_dists = []

    for doc, meta, dist in zip(
        raw["documents"][0],
        raw["metadatas"][0],
        raw["distances"][0]
    ):
        if any(k in doc for k in keywords):
            filtered_docs.append(doc)
            filtered_metas.append(meta)
            filtered_dists.append(dist)

    # if nothing matched, return original (fallback)
    if not filtered_docs:
        return raw

    return {
        "documents": [filtered_docs],
        "metadatas": [filtered_metas],
        "distances": [filtered_dists]
    }


In [14]:
def final_query(query):
    keywords = extract_keywords(query)

    # 1. Broad dense recall
    q_emb = embed_model.encode([query], normalize_embeddings=True).tolist()
    raw = collection.query(
        query_embeddings=q_emb,
        n_results=40
    )

    # 2. EARLY keyword anchoring
    raw = keyword_filter_raw(raw, keywords)

    # 3. Quality rerank
    reranked = rerank(raw)

    # 4. Section coherence
    final = filter_by_section(reranked)

    return final


In [15]:
final_query("मूर्खभृत्यस्य कथा")


[{'score': 0.64,
  'language': 'sa',
  'text': 'वरम् भृत्यविहिनस्य जिवितम् श्रमपूरितम् । मूर्खभृत्यस्य संसर्गात् सर्वम् कार्यम् विनश्यति ॥',
  'section_id': 1},
 {'score': 0.526, 'language': 'sa', 'text': 'मूर्खभृत्यस्य', 'section_id': 1}]